# This notebook demonstrates the application of Prescriptive Analytics - Causal Model using EconML and DoubleML

* EconML is part of Microsoft ALICE Project: https://www.microsoft.com/en-us/research/group/alice/

* And DoubleML: https://docs.doubleml.org/stable/index.html

## In addition, we will also use:

* SHAP algorithm to determine the magnitude and direction of influence
* Many Machine Learning algorithms to find the magnitude and direction of causal factors

# There are four types of data analytics:

##1. Descriptive: answers questions about “what happened?”
* These can be answered with statistics, data visualisations and interpretations


##2. Diagnostic: “why did this happen?”
* We won't treat this separately but we will look at it from the standpoint of descriptive, predictive and prescriptive

##3. Predictive: “what is going to happen in the future?” [forecasting & prediction]
* This will help us determine the data  (data variables, features, columns) that influences outcomes
* At times this approach will not identify causal factors, instead these variables may be correlated
* Features Importances - rank these in order of influence on the outcomes e.g., if we discover that certain data columns have minimal importance, the idea is that we can remove these from the "model".

##4. Prescriptive: “what actions should be taken next?”
* This is to identify causal factors

##All four types are often used together to create a complete story of based on data (data storytelling).

## Importantly, we want to combine both Predictive and Prescriptive approaches since Predictive will identify correlated features and Prescriptive will identify casual factors - together we will have identified all relevant data to create a complete "model"  (data plus machine learning algorithms).

#Terminology:

* Confounders (covariates): a confounding variable is an unmeasured variable that influences both the supposed cause and effect.

* Treatment variable: also called the independent variable (the one you think might be the cause) and then measure the dependent variable (the one you think might be the effect) to find out what this effect might be.


* Instrument variable: An instrumental variable is a third variable, Z, used in regression analysis when you have endogenous variables — variables that are influenced by other variables in the model. In other words, you use it to account for unexpected behavior between variables. Using an instrumental variable to identify the hidden (unobserved) correlation allows you to see the true correlation between the explanatory variable and response variable, Y [https://www.statisticshowto.com/instrumental-variable/]

In [1]:
# Step 1: Remove problematic packages built for NumPy 2.x
%pip uninstall -y shap scikit-learn scipy



Found existing installation: shap 0.43.0
Uninstalling shap-0.43.0:
  Successfully uninstalled shap-0.43.0
Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2
Found existing installation: scipy 1.15.2
Uninstalling scipy-1.15.2:
  Successfully uninstalled scipy-1.15.2


# Business Problem - Housing Affordability and Prices
## Does Race cause a decline in house prices?

## Sections:

##1. A Gentle Start: Linear Regression
##2. Train a Fine-tuned Predictive ML Model
##3. Correlation Interpretation
* Feature Importance -- Learn the top predictors for a given ML model
* Partial Dependence Plot -- Learn the statistical relationship between share of Black residents and housing price

##4. Causal Interpretation
* Direct Causal Effect -- Do the top predictors also have a direct effect on outcome of interest?

* Segmentation -- How different type of houses respond differently to number of rooms?

* What If Analysis -- How the overall housing price changes with one more room?
* Policy Analysis -- What is the best policy considering cost?
* Cohort Analysis -- What is the causal effect on a new dataset?

## Data Dictionary

* CRIM	per capita crime rate by town
* ZN	proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS	proportion of non-retail business acres per town.
* CHAS	Charles River dummy variable (1 if tract bounds river; 0 otherwise)
* NOX	nitric oxides concentration (parts per 10 million)
* RM	average number of rooms per dwelling
* AGE	proportion of owner-occupied units built prior to 1940
* DIS	weighted distances to five Boston employment centres
* RAD	index of accessibility to radial highways
* TAX	full-value property-tax rate per $10,000
* PTRATIO	pupil-teacher ratio by town
* B	is the proportion of Black residents by town
* LSTAT	lower socioeconomic status by town: (share of adults with less than high school education + share of male workers classified as laborers)

* MEDV	Median value of owner-occupied homes in $1000's

## Install and import python libraries as well as create settings

In [2]:
%pip install lightgbm==4.5.0 econml

  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached shap-0.43.0-cp311-cp311-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (24 kB)
Using cached scikit_learn-1.5.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
Using cached shap-0.43.0-cp311-cp311-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (532 kB)


In [3]:
import econml

from econml.orf import DMLOrthoForest, DROrthoForest
from econml.dml import CausalForestDML
from econml.sklearn_extensions.linear_model import WeightedLassoCVWrapper, WeightedLasso, WeightedLassoCV

In [4]:
# core python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [5]:
# view charts inline
%matplotlib inline

In [6]:
# machine learning
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.model_selection import GridSearchCV

In [7]:
# shap algorithm
import shap

## Load dataset

In [8]:
from sklearn.datasets import fetch_openml

boston_data = fetch_openml(data_id=531)
boston_data

{'data':         CRIM    ZN  INDUS CHAS    NOX     RM   AGE     DIS RAD    TAX  \
 0    0.00632  18.0   2.31    0  0.538  6.575  65.2  4.0900   1  296.0   
 1    0.02731   0.0   7.07    0  0.469  6.421  78.9  4.9671   2  242.0   
 2    0.02729   0.0   7.07    0  0.469  7.185  61.1  4.9671   2  242.0   
 3    0.03237   0.0   2.18    0  0.458  6.998  45.8  6.0622   3  222.0   
 4    0.06905   0.0   2.18    0  0.458  7.147  54.2  6.0622   3  222.0   
 ..       ...   ...    ...  ...    ...    ...   ...     ...  ..    ...   
 501  0.06263   0.0  11.93    0  0.573  6.593  69.1  2.4786   1  273.0   
 502  0.04527   0.0  11.93    0  0.573  6.120  76.7  2.2875   1  273.0   
 503  0.06076   0.0  11.93    0  0.573  6.976  91.0  2.1675   1  273.0   
 504  0.10959   0.0  11.93    0  0.573  6.794  89.3  2.3889   1  273.0   
 505  0.04741   0.0  11.93    0  0.573  6.030  80.8  2.5050   1  273.0   
 
      PTRATIO       B  LSTAT  
 0       15.3  396.90   4.98  
 1       17.8  396.90   9.14  
 2       

In [9]:
# data columns or features
boston_data.feature_names

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT']

In [10]:
# data for each column
boston_data.data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48


In [11]:
boston_data.data.shape

(506, 13)

In [12]:
# the house prices - what we are trying to  predict
# oftten called the outcome or independent variable
boston_data.target

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2
...,...
501,22.4
502,20.6
503,23.9
504,22.0


## Linear Regression

In [13]:
# add a constant column - we want to find the equation and intercept
X = sm.add_constant(boston_data.data)

In [14]:
X

,const,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98
1,1.0,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14
2,1.0,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03
3,1.0,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94
4,1.0,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,1.0,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67
502,1.0,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08
503,1.0,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64
504,1.0,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48


In [15]:
X.dtypes

,0
const,float64
CRIM,float64
ZN,float64
INDUS,float64
CHAS,category
NOX,float64
RM,float64
AGE,float64
DIS,float64
RAD,category


In [16]:
# Convert dataframe types
X_df = X.astype('float64')

#  rename column
X_df = X_df.rename(columns={"const": "Intercept"})

In [17]:
X_df

,Intercept,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,1.0,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,1.0,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,1.0,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,1.0,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,1.0,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,1.0,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,1.0,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,1.0,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


## Construct the model with linear regression

In [18]:
my_linear_regression =  sm.OLS(boston_data.target, X_df)

In [19]:
model = sm.OLS(boston_data.target, X_df)

In [20]:
model.__dict__

{'weights': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
 

## Train ML

In [21]:
results = model.fit()

In [22]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Mon, 07 Apr 2025   Prob (F-statistic):          6.72e-135
Time:                        01:09:15   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.000      26.432      46.487
CRIM          -0.1080      0.033     -3.287      0.001      -0.173      -0.043
ZN             0.0464      0.014      3.382      0.001       0.019       0.073
INDUS          0.0206      0.061      0.334      0.738      -0.100       0.141
CHAS           2.6867      0.862      3.118      0.002       0.994       4.380
NOX          -17.7666      3.820     -4.651      0.000     -25.272     -10.262
RM             3.8099      0.418      9.116      0.000       2.989       4.631
AGE            0.0007      0.013      0.052      0.958      -0.025       0.027
DIS           -1.4756      0.199     -7.398      0.000      -1.867      -1.084
RAD            0.3060      0.066      4.613      0.000       0.176       0.436
TAX           -0.0123      0.004     -3.280      0.001      -0.020      -0.005
PTRATIO       -0.9527      0.131     -7.283      0.000      -1.210      -0.696
B              0.0093      0.003      3.467      0.001       0.004       0.015
LSTAT         -0.5248      0.051    -10.347      0.000      -0.624      -0.425
==============================================================================
Omnibus:                      178.041   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              783.126
Skew:                           1.521   Prob(JB):                    8.84e-171
Kurtosis:                       8.281   Cond. No.                     1.51e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.51e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Interpretation: adjusted R-squared is the proportion of the outcome/target variable explained by the the features/predictors

## Predictive ML model

### split data into training (80%) and testing (20%)

In [23]:
# Split data into train and test
from sklearn.model_selection import train_test_split

# drop intercept column (not needed)
X_df = X_df.drop(columns=["Intercept"])

x_train, x_test, y_train, y_test = train_test_split(
    X_df, boston_data.target, test_size=0.2, random_state=0
)

In [24]:
categorical = ["CHAS"]
# Store the numerical columns in a list numerical
numerical = list(set(boston_data.feature_names).difference(set(categorical)))

In [25]:
numerical

['AGE',
 'B',
 'RM',
 'LSTAT',
 'DIS',
 'TAX',
 'INDUS',
 'ZN',
 'NOX',
 'PTRATIO',
 'RAD',
 'CRIM']

## create the model

In [26]:
# train a lightGBM regression model
ml_model = LGBMRegressor()

In [27]:
# establish the search paramters
param_grid = {"learning_rate": [0.1, 0.05, 0.01], "max_depth": [3, 5, 10]}

### Grid Search: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [28]:
# use grid searrch
search = GridSearchCV(ml_model, param_grid, n_jobs=-1)

In [ ]:
# now train  ML
search.fit(x_train, y_train)

In [ ]:
print("Best estimator: ", search.best_params_)

## Accuracy - use test data

In [ ]:
print("Test set score: ", search.best_estimator_.score(x_test, y_test))

#  Correlation Interpretation
## SHAP algorithm: https://shap.readthedocs.io/en/latest/index.html


In [ ]:
fitted_model = search.best_estimator_

In [ ]:
# use interventional approach
background = shap.maskers.Independent(x_train, max_samples=1000)

In [ ]:
background.__dict__

# Explainable Machine Learning (Model understanding) with SHAP

In [ ]:
explainer = shap.TreeExplainer(
    fitted_model, data=background, feature_names=boston_data.feature_names
)

In [ ]:
shap_values = explainer(x_test)

* CRIM	per capita crime rate by town
* ZN	proportion of residential land zoned for lots over 25,000 sq.ft.
* INDUS	proportion of non-retail business acres per town.
* CHAS	Charles River dummy variable (1 if tract bounds river; 0 otherwise)
* NOX	nitric oxides concentration (parts per 10 million)
* RM	average number of rooms per dwelling
* AGE	proportion of owner-occupied units built prior to 1940
* DIS	weighted distances to five Boston employment centres
* RAD	index of accessibility to radial highways
* TAX	full-value property-tax rate per $10,000
* PTRATIO	pupil-teacher ratio by town
* B	is the proportion of Black residents by town
* LSTAT	lower socioeconomic status by town: (share of adults with less than high school education + share of male workers classified as laborers)

* MEDV	Median value of owner-occupied homes in $1000's

In [ ]:
# plot the feature importance
shap.summary_plot(shap_values, x_test)

###  SHAP show  the most important features sorted by their importance level. It tells us that neighborhoods with a smaller shares of low socioeconomic status residents, higher median number of rooms and less pupil-teacher ratio will have a higher housing price.

###  This in line with what we learnt from Linear Regression above.

SHAP force plots

In [ ]:
shap.initjs()
shap.force_plot(shap_values)

In [ ]:
shap.force_plot(shap_values[0], matplotlib=True)

### Partial Dependence  Plot  - Show statistical relationship between share of Black residents and housing price

The top 5 features by correlation:

AGE, RM, NOX, CRIM, TAX

In [ ]:
shap.plots.partial_dependence(
    "B",
    fitted_model.predict,
    pd.DataFrame(x_test, columns=boston_data.feature_names),
    ice=False,
    model_expected_value=True,
    feature_expected_value=True,
)

In [ ]:
shap.plots.partial_dependence(
    "NOX",
    fitted_model.predict,
    pd.DataFrame(x_test, columns=boston_data.feature_names),
    ice=False,
    model_expected_value=True,
    feature_expected_value=True,
)

In [ ]:
shap.plots.partial_dependence(
    "TAX",
    fitted_model.predict,
    pd.DataFrame(x_test, columns=boston_data.feature_names),
    ice=False,
    model_expected_value=True,
    feature_expected_value=True,
)

### Interpretations: Taking share of Black residents as an example, here B is a function of Black population in town, the higher of B, the lower of Black population(%). From the coefficient of linear regression, the shap summary plot and also the partial dependence plot, we could get the same conclusion that there is a positive correlation between B and median housing price. In other word, housing price will decrease with the increasing of Black population(%). However, is that really causal? Let us validate that in the following section.

Note, this relationship is not linear. B = 1000(Bk - 0.63)^2 where Bk is the proportion. Hence a high value of B means low level of black population, however a high level of black population is a medium value.

###  Overall, all the insights above are coming from corelation perspective, telling us the positive or negative correlation between each predictor and the target.

### To correctly find the causal relationship, we have to train a different model controlling on all the possible hidden variables (confounders) and learn the direct causal effect for a given feature.

### That's what the causal interpretation tool is doing. In the following section, we will explore the causal relationship in different ways.

# Causal Interpretation
### Business question: Direct Causal Effect -- Do the top predictors also have a direct effect on outcome of interest?

In [ ]:
classification = False

In [ ]:
# order feature names according to shap values
vals = np.abs(shap_values.values).mean(0)
feature_importance = pd.DataFrame(
    list(zip(shap_values.feature_names, vals)), columns=["features", "importance"]
)

In [ ]:
feature_importance.sort_values(by=["importance"], ascending=False, inplace=True)

In [ ]:
top_features = feature_importance["features"]

In [ ]:
top_features

In [ ]:
from econml.solutions.causal_analysis import CausalAnalysis

In [ ]:
causal_analysis = CausalAnalysis(
    top_features,
    categorical,
    heterogeneity_inds=None,
    classification=classification,
    nuisance_models="automl",
    heterogeneity_model="linear",
    n_jobs=-1,
    random_state=123,
)

In [ ]:
causal_analysis.fit(pd.DataFrame(x_train, columns=boston_data.feature_names), y_train)

In [ ]:
# get global causal effect ordered by causal importance (pvalue)
global_summ = causal_analysis.global_causal_effect(alpha=0.05)
global_summ.sort_values(by="p_value")

## Visualise Causal Factors - Direction and Magnitude

In [ ]:
# helper function to plot error bar
def errorbar(res):
    xticks = res.index.get_level_values(0)
    lowererr = res["point"] - res["ci_lower"]
    uppererr = res["ci_upper"] - res["point"]
    xticks = [
        "{}***".format(t)
        if p < 1e-6
        else ("{}**".format(t) if p < 1e-3 else ("{}*".format(t) if p < 1e-2 else t))
        for t, p in zip(xticks, res["p_value"])
    ]
    plot_title = "Direct Causal Effect of Each Feature with 95% Confidence Interval, "
    plt.figure(figsize=(15, 5))
    plt.errorbar(
        np.arange(len(xticks)),
        res["point"],
        yerr=[lowererr, uppererr],
        fmt="o",
        capsize=5,
        capthick=1,
        barsabove=True,
    )
    plt.xticks(np.arange(len(xticks)), xticks, rotation=45)
    plt.title(plot_title)
    plt.axhline(0, color="r", linestyle="--", alpha=0.5)
    plt.ylabel("Average Treatment Effect")

##  The Average Treatment Effect (ATE) for each feature, assuming they are the treatment. The error bar above is ordered by feature importance, and the summary table above is ordered by causal significance (p-value).

In [ ]:
errorbar(global_summ)

## Findings:  share of Black residents (B), we could see it also gives us **insignificant causal effect on housing price**, which means race by itself has no direct causal effect on home prices. By learning the correlation between B and other features, we could see it's highly correlated with crime rate(CRIM) and percentage of lower status population (LSTAT), which do have strong causal effects. This pattern of correlations make B as a strong predictor but not a direct driver. Using the causal analysis tool has helped us avoid reaching a controversial and incorrect conclusion.

## Segmentation -- How different type of houses respond differently to number of rooms?

##CATE: Conditional Average Treatment Effect - https://en.wikipedia.org/wiki/Average_treatment_effect

###  From the global level, we know that the ATE of RM is 4.5, which means in average adding one more room will raise the housing price by 4.5 units. In the shallow tree above, we could see although overall RM has a significant positive effect on housing price, housing price will be more expensive for one more room in regions with lower pupil-teacher rate, and the effect will be insignificant in the regions with higher pupil-teacher rate and lower retail business rate.

In [ ]:
plt.figure(figsize=(12, 8))


causal_analysis.plot_heterogeneity_tree(
    pd.DataFrame(x_test, columns=boston_data.feature_names),
    "RM",
    max_depth=2,
    min_impurity_decrease=1e-6,
)

## Policy Analysis -- What is the best policy considering cost?

### The recommended policy  (if followed), on average, the housing price will increase by 2 more units compared with no more room added. Similarly, it will increase by around 1.4 units compared with adding one more room for every house. To be more detailed, we could also output the individualized policy. In the following table, I will only print the top five houses ordered by policy gains.

In [ ]:
plt.figure(figsize=(12, 8))

causal_analysis.plot_policy_tree(
    pd.DataFrame(x_test, columns=boston_data.feature_names),
    "RM",
    treatment_costs=8,
    max_depth=2,
)

## Note the effect of treatment is the treatment effect of increasing or decreasing 10% of average treatment level minus the cost, and decrease or increase mean in which direction we will get positive policy gain.

In [ ]:
causal_analysis.individualized_policy(
    pd.DataFrame(x_test, columns=boston_data.feature_names),
    "RM",
    n_rows=5,
    treatment_costs=4,
    alpha=0.1,
)

## What If Analysis - How the overall housing price changes with one more room?


In [ ]:
cf = causal_analysis.whatif(x_test, x_test['RM'] + 1, 5, y_test)

In [ ]:
print("Current average housing price on test set: ", y_test.mean())
print(
    "Average housing price with one more room on test set: ",
    cf["point_estimate"].mean(),
)

###  Summary table: we could see overall if we add one more room in the test set, the housing price will increase by 4+ units, which is in line with the ATE we learnt above. And the histrogram shows a comparison between the current housing price distribution and the counterfactuals ditribution if we add one more room.

In [ ]:
# distribution comparison
plt.hist(cf.point_estimate, label="With one more room", alpha=0.7)
plt.hist(y_test, label="Current", alpha=0.7)
plt.legend()
plt.xlabel("Housing Price")
plt.title("Histogram of Housing price -- Current vs. One more room")

## Cohort Analysis -- What is the causal effect on a new dataset?


In [ ]:
# global effect on new dataset
causal_analysis.cohort_causal_effect(x_test)

In [ ]:
# local effect on new dataset
causal_analysis.local_causal_effect(x_test)

#END WORKSHOP